In [ ]:
import os
import requests
import json
import pandas as pd

from dotenv import load_dotenv

from requests_oauthlib import OAuth1

In [ ]:
load_dotenv()

In [ ]:
ACCOUNT_ID = os.getenv('account')
CONSUMER_KEY = os.getenv('consumer_key')
CONSUMER_SECRET = os.getenv('consumer_secret')
TOKEN_ID = os.getenv('token_key')
TOKEN_SECRET = os.getenv('token_secret')

In [ ]:
auth = OAuth1(
    client_key=CONSUMER_KEY,
    client_secret=CONSUMER_SECRET,
    resource_owner_key=TOKEN_ID,
    resource_owner_secret=TOKEN_SECRET,
    realm=ACCOUNT_ID,
    signature_method='HMAC-SHA256'
)

In [ ]:
header_suiteql = {'Prefer': 'transient'}

In [ ]:
# limit = no of records to fetch in a api call (default and max =1000)
# offset = no of records to skip
# offset should be divisible by limit
url = f"https://{ACCOUNT_ID}.suitetalk.api.netsuite.com/services/rest/record/v1/account?limit=10&offset=100"
res = requests.get(url, auth=auth)
print(res.status_code)
print(res.text)

res_json = json.loads(res.text)
for key, val in res_json.items():
    if isinstance(val, list):
        print(f"{key}: {val[0]}")
    else:
        print(f"{key}: {val}") 

### Fetching Transaction Line

In [ ]:
query_transaction_line = 'SELECT * FROM transactionline'

In [ ]:
url_transaction_line = f'https://{ACCOUNT_ID}.suitetalk.api.netsuite.com/services/rest/query/v1/suiteql'
res_transaction_line = requests.post(url=url_transaction_line, 
                                    auth=auth,
                                    headers=header_suiteql,
                                    json={"q": query_transaction_line}
                                )
print(res_transaction_line.status_code)
print(res_transaction_line.text)

json_transaction_line = json.loads(res_transaction_line.text)
for key, val in json_transaction_line.items():
    if isinstance(val, list):
        print(f"{key}: {val[0]}")
    else:
        print(f"{key}: {val}")

df_transaction_line = pd.DataFrame(json_transaction_line['items'])
print(df_transaction_line.info())
display(df_transaction_line.head())

In [ ]:
def clean_text_columns(df):
    for col in df.select_dtypes(include='object'):
        df[col] = df[col].str.replace(r'[\n\r]', ' ', regex=True) # replace \n \r with white space
    return df

df_transaction_line = clean_text_columns(df_transaction_line)

In [ ]:
df_transaction_line.to_csv('netsuite_fetched_data/transaction_line.csv', index=False)